In [3]:
import numpy as np
import cv2

In [4]:
cap=cv2.VideoCapture(0)
ret,frame1=cap.read()
prvsImg=cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1]=255

while True:
    ret,frame2=cap.read()
    nextImg=cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow=cv2.calcOpticalFlowFarneback(prvsImg,nextImg,None,0.5,3,15,3,5,1.2,0)
    mag,ang=cv2.cartToPolar(flow[:,:,0],flow[:,:,1],angleInDegrees=True)
    hsv_mask[:,:,0]=ang/2
    hsv_mask[:,:,2]=cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    bgr=cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2',bgr)
    if cv2.waitKey(30) & 0xff == ord('q'):
        break
    prvsImg=nextImg
cv2.destroyAllWindows()
cap.release()

Bu kod, webcam'den görüntü alarak Farneback optik akış algoritması ile hareketi tespit eden ve bunu görselleştiren bir OpenCV uygulamasıdır. Her bir bölümünü ayrıntılı olarak açıklayalım:

### Gerekli Kütüphanelerin Yüklenmesi
```python
import cv2
import numpy as np
```
- **cv2**: OpenCV kütüphanesi, bilgisayarla görme uygulamaları için kullanılır.
- **numpy (np)**: Sayısal hesaplamalar ve dizi işlemleri için kullanılır.

### Video Yakalama ve İlk Kareyi Alma
```python
cap = cv2.VideoCapture(0)
ret, frame1 = cap.read()
prvsImg = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
```
- **cap = cv2.VideoCapture(0)**: Bilgisayarın kamerasından video yakalamak için bir nesne oluşturur (0, varsayılan kamera anlamına gelir).
- **ret, frame1 = cap.read()**: Kameradan bir kare yakalar. `ret` değişkeni, yakalamanın başarılı olup olmadığını kontrol etmek için kullanılır.
- **prvsImg = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)**: İlk kareyi gri tonlamaya dönüştürür. Optik akış hesaplamaları genellikle gri tonlamalı görüntüler üzerinde yapılır.

### HSV Maske Tanımlaması
```python
hsv_mask = np.zeros_like(frame1)
hsv_mask[:, :, 1] = 255
```
- **hsv_mask = np.zeros_like(frame1)**: İlk kare ile aynı boyutlarda sıfırlarla dolu bir maske oluşturur.
- **hsv_mask[:, :, 1] = 255**: HSV (Hue, Saturation, Value) renk uzayında 'saturation' (doygunluk) kanalını maksimum (255) yapar. Bu, renklerin tam doygunlukta olmasını sağlar.

### Ana Döngü
```python
while True:
    ret, frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
```
- **ret, frame2 = cap.read()**: Kameradan bir sonraki kareyi yakalar.
- **nextImg = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)**: Yeni kareyi gri tonlamaya dönüştürür.

### Optik Akış Hesaplaması
```python
    flow = cv2.calcOpticalFlowFarneback(prvsImg, nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[:, :, 0], flow[:, :, 1], angleInDegrees=True)
```
- **flow**: Farneback yöntemi kullanılarak optik akışı hesaplar. Bu, önceki ve sonraki kareler arasındaki piksel hareketlerini tahmin eder.
    - **0.5**: Piramit ölçek faktörü.
    - **3**: Piramit seviyeleri.
    - **15**: Pencere boyutu.
    - **3**: Ortalama iterasyon sayısı.
    - **5**: Poly-n boyutu.
    - **1.2**: Poly-sigma.
    - **0**: Flags (0 ile standart Farneback kullanımı).
- **mag, ang = cv2.cartToPolar(flow[:, :, 0], flow[:, :, 1], angleInDegrees=True)**: Kartesyen (x, y) optik akış vektörlerini polar (büyüklük ve açı) koordinatlara dönüştürür.

### HSV Maske Güncellemesi
```python
    hsv_mask[:, :, 0] = ang / 2
    hsv_mask[:, :, 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
```
- **hsv_mask[:, :, 0] = ang / 2**: Açıyı HSV renk uzayında 'hue' kanalına yerleştirir. Açılar 0-360 derece arasında olduğu için 0-180 derece arasına normalize edilir.
- **hsv_mask[:, :, 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)**: Büyüklüğü normalize edip 'value' kanalına yerleştirir.
- **bgr = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)**: HSV maskesini BGR formatına dönüştürür.

### Görüntüleme ve Döngüden Çıkış
```python
    cv2.imshow('frame2', bgr)
    if cv2.waitKey(30) & 0xff == ord('q'):
        break
    prvsImg = nextImg
```
- **cv2.imshow('frame2', bgr)**: BGR formatındaki optik akış görüntüsünü gösterir.
- **if cv2.waitKey(30) & 0xff == ord('q')**: 30 milisaniye bekler ve 'q' tuşuna basılırsa döngüden çıkar.
- **prvsImg = nextImg**: Sonraki kareyi önceki kare olarak günceller, böylece bir sonraki döngüde kullanılabilir.

### Temizlik
```python
cv2.destroyAllWindows()
cap.release()
```
- **cv2.destroyAllWindows()**: Tüm OpenCV pencerelerini kapatır.
- **cap.release()**: Kamera kaynağını serbest bırakır.

Bu kod, gerçek zamanlı olarak kameradan alınan görüntülerdeki hareketleri tespit etmek ve bu hareketleri görselleştirmek için Farneback optik akış algoritmasını kullanır.